# STEP 0

## 0.1. Imports


In [1]:
import pandas                   as pd
import seaborn                  as sns
import sweetviz                 as sv
import matplotlib.pyplot        as plt
import seaborn                  as sns
import xgboost                  as xgb
import lightgbm                 as lgb
from sklearn.svm                import SVC 
from sklearn                    import metrics
from sklearn.pipeline           import Pipeline
from sklearn.model_selection    import GridSearchCV
# from sklearn.impute             import SimpleImputer
# from sklearn.model_selection    import cross_validate
from sklearn.model_selection    import train_test_split
from sklearn.compose            import ColumnTransformer
from sklearn.model_selection    import StratifiedKFold, KFold
from sklearn.linear_model       import LogisticRegression
from sklearn.preprocessing      import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing      import LabelEncoder, OneHotEncoder, OrdinalEncoder

c:\Users\Usuário\repos\google_ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0.2. Helper Functions

In [2]:
import warnings
warnings.filterwarnings("ignore")

# ===================================================================
# Configura os gráficos
def jupyter_settings():
    %matplotlib inline
    # %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [22, 9]
    plt.rcParams['font.size'] = 21

    # display(HTML('<style>.conteiner{width:100% !important;}</style>'))

    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    # configura o pandas para quantidade de casas decimais
    pd.set_option('display.float_format', lambda x: '%.2f' % x)

    sns.set()

jupyter_settings()

# Calcula e retorna um dataframe com as métricas sem validação cruzada
def simple_metrics(model_name, test, predict):
    data = [
        [
            model_name,
            metrics.precision_score(test, predict),
            metrics.recall_score(test, predict),
            metrics.f1_score(test, predict),
            metrics.roc_auc_score(test, predict),
        ]
    ]
    columns = ["Model", "Precision", "Recall", "F1", "AUC"]
    metrics_table = pd.DataFrame(data, columns=columns)
    return metrics_table


def tuning_hyperparams(pre_processor, modelo, model_name, param, X, y):
    '''
        Método que utiliza o Pipeline para pré processar os dados e calcula quais são 
        os melhores hiperparâmetros baseado no modelo e no dicionário de hiperparâmetros 
        informados. Retorna um dicionário com os parâmetros testados e os melhores valores
        de cada um.

        pre_processor: objeto da classe ColumnTransformer escolhido para os dados
        modelo: instância do algoritimo a ser usado
        model_name: String com o apelido do modelo a ser usado
        param: Dict - dicionário com os hiperparams e os valores a serem testados
        X: dados de treino
        y: variável target dos dados de treino

        return dict
    '''
    # criando o modelo usando pipeline
    model = Pipeline(
        steps=[
            ("preprocessor", pre_processor),
            (model_name, modelo),
        ]
    )

    # Criando o dicionario de hiperparâmetros
    test_keys = [model_name + '__' + x for x in param.keys()] # colocar duplo underscore entre o nome do modelo e o nome do parâmetro  (lr__)
    test_values = list(param.values())
    parameters = { test_keys[i]: test_values[i] for i in range(len(test_keys)) }

    # Rodando 5-fold cross-validation com gridsearch
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    grid = GridSearchCV(
        model, param_grid=parameters, cv=kfold, n_jobs=-1, return_train_score=True
    )

    grid.fit(X=X, y=y)

    # Imprime os melhores parâmetros
    param = grid.best_params_
    return param

## 0.3. Loading Data

In [3]:
test_raw = pd.read_csv('../data/test.csv')
train_raw = pd.read_csv('../data/train.csv')

# 1 - Data understanding


In [4]:
df = train_raw.copy()
df.head(3)

id_do_caso continente educacao_do_empregado tem_experiencia_de_trabalho requer_treinamento_de_trabalho  num_de_empregados  ano_de_estabelecimento regiao_de_emprego  salario_prevalecente unidade_de_salario posicao_em_tempo_integral status_do_caso
0  EZYV10567     Europa          Ensino Médio                           N                              S               2087                    1855               Sul              69711.24                Ano                         S         Negado
1   EZYV5505       Ásia              Mestrado                           S                              N               5991                    2003        Meio-Oeste              52931.38                Ano                         S       Aprovado
2   EZYV5207       Ásia          Ensino Médio                           N                              N               1426                    2000              Ilha             110830.21                Ano                         S         Negado

## 1.1. Rename Columns

In [5]:
df.columns = ['id', 'continente', 'escolaridade',
       'tem_experiencia', 'requer_treinamento',
       'num_empregados', 'ano_fundacao', 'regiao',
       'salario_medio', 'periodicidade',
       'tempo_integral', 'status']

## 1.2. Data Dimensions

In [6]:
# Conferindo a volumetria
print(f"Quantidade de linhas:   {df.shape[0]}")
print(f"Quantidade de colunas:  {df.shape[1]}")
print(f"IDs únicos:             {df.id.nunique()}\n")

Quantidade de linhas:   17836
Quantidade de colunas:  12
IDs únicos:             17836



## 1.3. Data Types

In [7]:
df.dtypes

id                     object
continente             object
escolaridade           object
tem_experiencia        object
requer_treinamento     object
num_empregados          int64
ano_fundacao            int64
regiao                 object
salario_medio         float64
periodicidade          object
tempo_integral         object
status                 object
dtype: object

## 1.4. Change Data Types

In [8]:
'''
Alterando os tipos de int64 e float64 para int32 e float32 respectivamente com o 
intuito de melhorar a performance além de alterar as variáveis nominais para o tipo 
category que consome menos memória que o tipo object.
'''
df.id = df.id.astype('string')
df.continente = df.continente.astype('category')
df.escolaridade = df.escolaridade.astype('category')
df.tem_experiencia = df.tem_experiencia.astype('category')
df.requer_treinamento = df.requer_treinamento.astype('category')
df.num_empregados = df.num_empregados.astype('int32')
df.ano_fundacao = df.ano_fundacao.astype('int32')
df.regiao = df.regiao.astype('category')
df.salario_medio = df.salario_medio.astype('float32')
df.periodicidade = df.periodicidade.astype('category')
df.tempo_integral = df.tempo_integral.astype('category')
df.status = df.status.apply(lambda status: 0 if status == 'Negado' else 1)
df.status = df.status.astype('int32')

## 1.5. Check NA


In [9]:
df.isnull().mean()

id                   0.00
continente           0.00
escolaridade         0.00
tem_experiencia      0.00
requer_treinamento   0.00
num_empregados       0.00
ano_fundacao         0.00
regiao               0.00
salario_medio        0.00
periodicidade        0.00
tempo_integral       0.00
status               0.00
dtype: float64

## 1.6. Descriptive Statistical


In [10]:
# Separando os atributos entre numéricos e categóricos
numerical_attributes = df.select_dtypes(include=['int32', 'float32'])
categorical_attributes = df.select_dtypes(exclude=['int32', 'float32'])

### 1.6.1 Numerical Features

In [11]:
numerical_attributes.describe().T

count     mean      std     min      25%      50%       75%       max
num_empregados 17836.00  5726.50 23321.38  -26.00  1023.00  2117.00   3504.25 602069.00
ano_fundacao   17836.00  1979.49    42.10 1800.00  1976.00  1997.00   2005.00   2016.00
salario_medio  17836.00 74327.42 52795.86    2.14 33892.91 70106.92 107564.71 319210.28
status         17836.00     0.67     0.47    0.00     0.00     1.00      1.00      1.00

### 1.6.2 Categorical Features

In [12]:
display( pd.DataFrame( categorical_attributes.apply(lambda x: x.unique().shape[0]), columns=[ 'Valores únicos'], ).sort_values('Valores únicos'))

Valores únicos
tem_experiencia                  2
requer_treinamento               2
tempo_integral                   2
escolaridade                     4
periodicidade                    4
regiao                           5
continente                       6
id                           17836

### 1.7 Target Feature

In [13]:
df.status.value_counts()

1    11937
0     5899
Name: status, dtype: int64

# 4 - Data preparation

In [14]:
df2 = df.copy()

# dividindo em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df2.drop(columns=["id", "status"], axis=1), df2["status"], test_size=0.2, random_state=42
)

# ======================== PRÉ PROCESSAMENTO ===============================
def pre_processa(data):
    df_pre = data.copy()
    # pipeline para pré-processamento das variáveis binárias
    for col in df_pre[['tem_experiencia', 'requer_treinamento', 'tempo_integral']].columns:
        df_pre[col] = df_pre[col].apply(lambda x: 0 if x=='N' else 1)

    # Criar feature de salário mensal
    df_pre['salario_medio_mes'] = df_pre.apply( lambda linha: 
                                        linha['salario_medio'] / 12 if (linha['periodicidade']=='Ano') else 
                                        linha['salario_medio'] if (linha['periodicidade']=='Mês') else 
                                        linha['salario_medio'] * 4         if (linha['periodicidade']=='Semana') else 
                                        linha['salario_medio'] * 8 * 30    if ((linha['periodicidade']=='Hora') & (linha['tempo_integral']==1)) 
                                        else linha['salario_medio'] * 4 * 30    if ((linha['periodicidade']=='Hora') & (linha['tempo_integral']==0)) 
                                        else 0, 
                                        axis=1)
    df_pre.drop(columns=['salario_medio', 'periodicidade'], inplace=True)

    # Corrigir empresas com qtd de funcionários negativos
    df_pre.num_empregados = df_pre.num_empregados.apply( lambda x: df_pre.num_empregados.median() if x < 1 else x)

    # pipeline para pré-processamento das variáveis categóricas
    ord_enc = OrdinalEncoder()
    df_pre[['continente', 'escolaridade', 'ano_fundacao', 'regiao', ]] = ord_enc.fit_transform(df_pre[['continente', 'escolaridade', 'ano_fundacao', 'regiao', ]])

    return df_pre

# Pré processa os dados
X_train = pre_processa(X_train)
X_test = pre_processa(X_test)
df3 = pre_processa( df2 )

# pipeline para pré-processamento das variáveis categóricas
scale_transformer = Pipeline(steps=[("scaler", StandardScaler())])

# Compondo os pré-processadores
preprocessor = ColumnTransformer(
    transformers=[
        ( "mmx", scale_transformer, list( df3.drop(columns=["id", "status"])),)
    ]
)

# 5 - Modeling

## 5.1. Logistic Regression

In [15]:
model_name = 'lr'
modelo = LogisticRegression()
parametros = { 
    "max_iter": [1, 5, 10],
    "penalty": ["l2", "l1"],
    "class_weight": ["balanced", None],
    'solver':['liblinear']
}

# O cross validation para descobrir os melhores parametros é realizado no dataset completo
best_params_lr = tuning_hyperparams(preprocessor, modelo, model_name, parametros, df3.drop(columns=['id', 'status']), df3["status"])

# Usando os melhores parâmetros do Cross validation
model_lr = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "lr",
            LogisticRegression(
                class_weight    = best_params_lr[model_name + '__' + "class_weight"],
                max_iter        = best_params_lr[model_name + '__' + "max_iter"],
                penalty         = best_params_lr[model_name + '__' + "penalty"],
                solver          = best_params_lr[model_name + '__' + "solver"],
            ),
        ),
    ]
)

# treinando o modelo
model_lr.fit(X_train, y_train)

# testando o modelo
y_pred = model_lr.predict(X_test)

# Salvando os melhores parametros
with open('../models/lr.pickle', 'wb') as handle:
    pickle.dump(best_params_lr, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Imprimindo as métricas
lr_metrics_table = simple_metrics('Logistic Regression', y_test, y_pred)
lr_metrics_table

Model  Precision  Recall   F1  AUC
0  Logistic Regression       0.70    0.93 0.80 0.55

In [ ]:
print(best_params_lr)

## 5.2. Suport Vector Machine

In [16]:
model_name = 'svm'
modelo = SVC(kernel='linear', C=2, gamma=1)

parametros = { 
    "C": [1.0, 3.0],
    "kernel": ['linear', 'rbf', 'sigmoid'],
    'gamma':['scale', 'auto'],
    "class_weight": ["balanced"],
}

# O cross validation para descobrir os melhores parametros é realizado no dataset completo
best_params_svm = tuning_hyperparams(preprocessor, modelo, model_name, parametros, df3.drop(["id", "status"], axis=1), df3["status"])

# Usando os melhores parâmetros do Cross validation
model_svm = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "svm",
            SVC(
                class_weight    = best_params_svm[model_name + '__' + "class_weight"],
                C               = best_params_svm[model_name + '__' + "C"],
                gamma           = best_params_svm[model_name + '__' + "gamma"],
                kernel          = best_params_svm[model_name + '__' + "kernel"],
            ),
        ),
    ]
)

# treinando o modelo
model_svm.fit(X_train, y_train)

# testando o modelo
y_pred = model_svm.predict(X_test)

# Salvando os melhores parametros
with open('../models/svm.pickle', 'wb') as handle:
    pickle.dump(best_params_svm, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Imprimindo as metricas
svm_metrics_table = simple_metrics('SVM', y_test, y_pred)
svm_metrics_table

Model  Precision  Recall   F1  AUC
0   SVM       0.82    0.75 0.78 0.70

In [ ]:
print(best_params_svm)

## 5.3. XGBoost

In [17]:
model_name = 'xgboost'

modelo = xgb.XGBClassifier(
    n_estimators=10000,
    objective="binary:logistic",
    tree_method="hist",
    eta=0.1,
    max_depth=3,
    random_state=0,
)

parametros = { 
    "n_estimators": [1000, 10000, 20000],
    "objective": ["binary:logistic","binary:logitraw", "binary:hinge"],
    'tree_method':['hist', 'approx'],
    "eta": [0.1, 0.5, 0.7],
    "max_depth":[2, 3, 5],
    "random_state":[42]
}

# O cross validation para descobrir os melhores parametros é realizado no dataset completo
best_params_xgb = tuning_hyperparams(preprocessor, modelo, model_name, parametros, df3.drop(["id", "status"], axis=1), df3["status"])

# Usando os melhores parâmetros do Cross validation
model_xgb = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "xbg",
            xgb.XGBClassifier(
                n_estimators    = best_params_xgb[model_name + '__' + "n_estimators"],
                objective       = best_params_xgb[model_name + '__' + "objective"],
                tree_method     = best_params_xgb[model_name + '__' + "tree_method"],
                eta             = best_params_xgb[model_name + '__' + "eta"],
                max_depth       = best_params_xgb[model_name + '__' + "max_depth"],
                random_state    = best_params_xgb[model_name + '__' + "random_state"],
            ),
        ),
    ]
)

# treinando o modelo
model_xgb.fit(X_train, y_train)

# testando o modelo
y_pred = model_xgb.predict(X_test)

# Salvando os melhores parametros
with open('../models/xgb.pickle', 'wb') as handle:
    pickle.dump(best_params_xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Imprimindo as métricas
xgb_metrics_table = simple_metrics('xgboost', y_test, y_pred)
xgb_metrics_table

Model  Precision  Recall   F1  AUC
0  xgboost       0.78    0.87 0.83 0.68

In [ ]:
print(best_params_xgb)

## 5.4. LGBM

In [18]:
model_name = 'lgbm'

modelo = lgb.LGBMClassifier()

parametros = { 
    "class_weight": ['balanced'],
    'objective':['binary'],
    "boosting_type": ['gbdt', 'dart', 'rf'],
    "n_estimators":[100, 200],
    "max_depth":[-1, 10, 20],
    "random_state":[42]
}

# O cross validation para descobrir os melhores parametros é realizado no dataset completo
best_params_lgbm = tuning_hyperparams(preprocessor, modelo, model_name, parametros, df3.drop(["id", "status"], axis=1), df3["status"])

# Usando os melhores parâmetros do Cross validation
model_lgbm = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "lgbm",
            lgb.LGBMClassifier(
                n_estimators    = best_params_lgbm[model_name + '__' + "n_estimators"],
                objective       = best_params_lgbm[model_name + '__' + "objective"],
                class_weight    = best_params_lgbm[model_name + '__' + "class_weight"],
                boosting_type   = best_params_lgbm[model_name + '__' + "boosting_type"],
                max_depth       = best_params_lgbm[model_name + '__' + "max_depth"],
                random_state    = best_params_lgbm[model_name + '__' + "random_state"],
            ),
        ),
    ]
)

# treinando o modelo
model_lgbm.fit(X_train, y_train)

# testando o modelo
y_pred = model_lgbm.predict(X_test)

# Salvando os melhores parametros
with open('../models/lgbm.pickle', 'wb') as handle:
    pickle.dump(best_params_lgbm, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Imprimindo as métricas
lgbm_metrics_table = simple_metrics('lgbm', y_test, y_pred)
lgbm_metrics_table

Model  Precision  Recall   F1  AUC
0  lgbm       0.83    0.74 0.78 0.71

In [ ]:
print(best_params_lgbm)

## Salvando os modelos

In [19]:
import pickle

with open('../models/lr.pickle', 'wb') as handle:
    pickle.dump(best_params_lr, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../models/svm.pickle', 'wb') as handle:
    pickle.dump(best_params_svm, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../models/xgb.pickle', 'wb') as handle:
    pickle.dump(best_params_xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../models/lgbm.pickle', 'wb') as handle:
    pickle.dump(best_params_lgbm, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Metrics

In [20]:
metrics_table = pd.concat([lr_metrics_table, svm_metrics_table, xgb_metrics_table, lgbm_metrics_table], ignore_index=True)
metrics_table

Model  Precision  Recall   F1  AUC
0  Logistic Regression       0.70    0.93 0.80 0.55
1                  SVM       0.82    0.75 0.78 0.70
2              xgboost       0.78    0.87 0.83 0.68
3                 lgbm       0.83    0.74 0.78 0.71